# Debiasing Documentation

In [1]:
import pprint
from debiasing.llm.models import OpenAICompletion
from debiasing.llm.models import AntrophicCompletion
from debiasing.llm.models import ModelConfigs
from debiasing.llm.utils import LLMMessage
from debiasing.llm.tools import (CALCULATOR,
                                 GENDER_BIAS_CLASSIFIER_DESCRIPTION, 
                                 GENDER_BIAS_MULTI_LABEL_CLASSIFIER,
                                 MultiLabelGenderBiasClassifier,)


## Use OpenAI and Antrophic chatcompletion models 

In [2]:
openai = OpenAICompletion()

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of France?"
    )
]

text, _, response = openai.get_answer(
    messages=msgs
)

print(text)

type='text' text='The capital of France is Paris.'


By default the LLM are instantiziate using the model specify in `config.py`, however you can provide the `model_id` with the LLM provider option:

In [3]:
# Ref: Antrophic models https://docs.anthropic.com/en/docs/about-claude/models
# Es posible instanciar un modelo de AntrophicCompletion con el model_id de un modelo específico
antrophic = AntrophicCompletion(model_id='claude-3-opus-20240229')

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of Chile?"
    )
]

text, _, response = antrophic.get_answer(
    messages=msgs
)

print(text)

type='text' text='The capital of Chile is Santiago.'


For using a system prompt, you can pass when instantiate the LLM.

In [4]:
llm_with_system_prompt = OpenAICompletion(
    system="Your are a joker and you have a very funny talkative style"
)

llm_without_system_prompt = OpenAICompletion(
    system=None
)

msgs = [LLMMessage(role=LLMMessage.MessageRole.USER, content="How are u?")]

print(llm_with_system_prompt.get_answer(messages=msgs)[0])
print('-' * 80)
print(llm_without_system_prompt.get_answer(messages=msgs)[0])

type='text' text="Oh, I'm doing just dandy, thank you for asking! I'm like a digital jester, always ready to juggle some words and tickle your funny bone. How about you? Are you ready for some giggles and guffaws today?"
--------------------------------------------------------------------------------
type='text' text="Thank you for asking! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"


In [5]:
llm_with_system_prompt = AntrophicCompletion(
    system="You are a seasoned data scientist at a Fortune 500 company."
)

llm_without_system_prompt = AntrophicCompletion(
    system=None
)

msgs = [LLMMessage(role=LLMMessage.MessageRole.USER, content="Introduce yourself!")]

print(llm_with_system_prompt.get_answer(messages=msgs)[0])
print('-' * 80)
print(llm_without_system_prompt.get_answer(messages=msgs)[0])

type='text' text="Hello! I'm an experienced data scientist with over a decade of experience working on enterprise-scale analytics and machine learning projects. In my current role at a Fortune 500 company, I focus on leveraging data science to drive business value through predictive modeling, optimization, and advanced analytics solutions.\n\nI have extensive experience with Python, R, SQL, and various ML frameworks, and I've led numerous successful projects in areas like customer analytics, supply chain optimization, risk modeling, and marketing attribution. I'm passionate about translating complex technical concepts into actionable business insights and helping organizations make data-driven decisions.\n\nHow can I help you today with your data science questions or challenges?"
--------------------------------------------------------------------------------
type='text' text="Hi! I'm Claude, an AI assistant created by Anthropic. I aim to be direct and honest in my interactions. I'm ha

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

In [7]:
print(prompt)

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of F

## Function Calling and Structured Outputs in OpenAI models

Ref: https://openai.com/index/introducing-structured-outputs-in-the-api/


The structured output mode is to order the LLM to complete given a user interaction a standard JSON form. 
There is two ways to use structured output using OpenAI model: 

1. Using via function calling mode as a new tool 
1. Dedicated structured output using `response_format`


We will use the first way which its advantage is instance an LLM with multiple tools allowing to determine which tool used in different context.

> _"When Structured Outputs is turned on, the arguments generated by the model for function calls will reliably match the JSON Schema that you provide_", [ref best practices section on function calling doc](https://platform.openai.com/docs/guides/function-calling#turn-on-structured-outputs-by-setting-strict-true)


In [8]:
gender_bias_example = (
    "La karencita es tan tierna. "
    "Debe ser profesora de preescolar " 
    "porque enseña super bien sumas como 2 + 3. "
    "Además no creo que sea ingenieria " 
    "porque a las mujeres no les gusta eso"
)

gender_bias_example2 = "Policeman are rude and violent"

calculator_example = "How much is 4 * 7 / 3?"

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        # content=calculator_example,
        content=gender_bias_example
    )
]

model_config = ModelConfigs(
    max_tokens=400,
    temperature=0.8,
)

openai = OpenAICompletion(configs=model_config, 
                          system=GENDER_BIAS_CLASSIFIER_DESCRIPTION,
                          tools=[GENDER_BIAS_MULTI_LABEL_CLASSIFIER, CALCULATOR])

text, tool, o_response = openai.get_answer(msgs)

print("text:", text)
print("tool:", tool)
o_response

text: None
tool: type='tool' name='gender_bias_classifier' arguments={'bias_label': ['STEREOTYPING_BIAS', 'SEXISM'], 'bias_text': ['Debe ser profesora de preescolar', 'no creo que sea ingenieria porque a las mujeres no les gusta eso'], 'score_label': [0.9, 0.95]}


{'id': 'chatcmpl-AhUWqQtQ0ZTpUoNK4rqk2f1oTnKor',
 'object': 'chat.completion',
 'created': 1734928820,
 'model': 'gpt-4o-2024-08-06',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_22xEcQatT00OtBv6H5of8pc0',
      'type': 'function',
      'function': {'name': 'gender_bias_classifier',
       'arguments': '{"bias_label":["STEREOTYPING_BIAS","SEXISM"],"bias_text":["Debe ser profesora de preescolar","no creo que sea ingenieria porque a las mujeres no les gusta eso"],"score_label":[0.9,0.95]}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 747,
  'completion_tokens': 66,
  'total_tokens': 813,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'system_fingerprint': 'fp_5f20662549'}

In [9]:
antrophic = AntrophicCompletion(configs=model_config,
                                system=GENDER_BIAS_CLASSIFIER_DESCRIPTION,
                                tools=[GENDER_BIAS_MULTI_LABEL_CLASSIFIER, CALCULATOR])

text, tool, a_response = antrophic.get_answer(msgs)

print("text", text)
print("tool", tool)
a_response

text type='text' text="I'll help you analyze this text for gender biases and also calculate the sum mentioned. Let me break this down:\n\n1. First, let's identify potential gender biases in the text:"
tool type='tool' name='gender_bias_classifier' arguments={'bias_label': ['STEREOTYPING_BIAS', 'SEMANTIC_BIAS', 'SEXISM'], 'bias_text': ['Debe ser profesora de preescolar', 'La karencita es tan tierna', 'no creo que sea ingenieria porque a las mujeres no les gusta eso'], 'score_label': [0.9, 0.85, 1.0]}


{'id': 'msg_01SPESkHCMjG3pjAULeo2u1J',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text',
   'text': "I'll help you analyze this text for gender biases and also calculate the sum mentioned. Let me break this down:\n\n1. First, let's identify potential gender biases in the text:"},
  {'type': 'tool_use',
   'id': 'toolu_01Q136ERLaJTgmvCquP3A9Xm',
   'name': 'gender_bias_classifier',
   'input': {'bias_label': ['STEREOTYPING_BIAS', 'SEMANTIC_BIAS', 'SEXISM'],
    'bias_text': ['Debe ser profesora de preescolar',
     'La karencita es tan tierna',
     'no creo que sea ingenieria porque a las mujeres no les gusta eso'],
    'score_label': [0.9, 0.85, 1.0]}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 1435,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 212}}

### Structured output with function calling

Direct request:

In [2]:

import requests
from debiasing.configs import settings


tools = [GENDER_BIAS_MULTI_LABEL_CLASSIFIER, CALCULATOR]
force_tool = False

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="La karencita es tan tierna. Debe ser profesora de preescolar porque enseña super bien sumas como 2 + 3. Además no creo que sea ingenieria porque a las mujeres no les gusta eso"
    )
]

parsed_messages = [
    {"role": message.role.value, "content": message.content}
    for message in msgs
]

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {settings.OPENAI_API_KEY}",
}


tool_config = (
    [tool.openai_dump() for tool in tools] if tools else None
)

pprint.pprint(tool_config)

body = {
    "model": "gpt-4o-2024-08-06",
    "messages": parsed_messages,
    "max_completion_tokens": 500,
    "temperature": 0.0,
    **(
        {
            "tools": tool_config,
            "tool_choice": "required" if force_tool else "auto",
        }
        if tool_config
        else {}
    )
}
body

[{'function': {'description': 'Identify gender biases in a given text (if any) '
                              'and for each bias specify in which specific '
                              'part of the text is located. Therefore, there '
                              'is a correspondence between the bias_label and '
                              'the bias_text.',
               'name': 'gender_bias_classifier',
               'parameters': {'$defs': {'GenderBiasesEnum': {'description': 'Kind '
                                                                            'of '
                                                                            'gender '
                                                                            'biases '
                                                                            'to '
                                                                            'detect '
                                                                            'in 

{'model': 'gpt-4o-2024-08-06',
 'messages': [{'role': 'user',
   'content': 'La karencita es tan tierna. Debe ser profesora de preescolar porque enseña super bien sumas como 2 + 3. Además no creo que sea ingenieria porque a las mujeres no les gusta eso'}],
 'max_completion_tokens': 500,
 'temperature': 0.0,
 'tools': [{'type': 'function',
   'function': {'name': 'gender_bias_classifier',
    'description': 'Identify gender biases in a given text (if any) and for each bias specify in which specific part of the text is located. Therefore, there is a correspondence between the bias_label and the bias_text.',
    'strict': True,
    'parameters': {'$defs': {'GenderBiasesEnum': {'description': 'Kind of gender biases to detect in the text\nRef: Based on Table 2 from work https://arxiv.org/pdf/2201.08675',
       'enum': ['GENERIC_PRONOUNS',
        'STEREOTYPING_BIAS',
        'SEXISM',
        'EXCLUSIONARY_TERMS',
        'SEMANTIC_BIAS'],
       'title': 'GenderBiasesEnum',
       'type':

In [ ]:
out = requests.post(
    settings.OPENAI_CHAT_ENDPOINT,
    headers=headers,
    json=body,
    timeout=settings.LLM_TIMEOUT
)

out.json()

{'id': 'chatcmpl-AguHYllG2xWznfX1xbdHgw3MXfupI',
 'object': 'chat.completion',
 'created': 1734789488,
 'model': 'gpt-4o-2024-08-06',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_GmX2UDGKzqXWkqKFeGsZq3Up',
      'type': 'function',
      'function': {'name': 'gender_bias_classifier',
       'arguments': '{"bias_label":["STEREOTYPING_BIAS","SEXISM"],"bias_text":["Debe ser profesora de preescolar porque enseña super bien sumas como 2 + 3.","no creo que sea ingenieria porque a las mujeres no les gusta eso"],"score_label":[0.9,0.95]}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 437,
  'completion_tokens': 79,
  'total_tokens': 516,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'system